In [1]:
import os
os.chdir("./project/nmos_inference")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from utils.data import df_split, df_balance, make_transistors_loader
from utils.util import  fix_all_seeds
from utils.trainer import Trainer
import yaml

ModuleNotFoundError: No module named 'utils'

## Data Sanity Check

In [6]:
df = pd.read_csv("./data_v3/full_balanced_2class_42/train_balanced_42.csv", index_col=0)
df["label"].value_counts()

1    448061
0    447787
Name: label, dtype: int64

## Model Sanity Check

In [7]:
config = yaml.safe_load(open("./config/config.yaml"))
data = np.load("./data_v1/original_3510_400.npy", mmap_mode='r')
df = pd.read_csv("./data_v3/train.csv")
df = df_balance(df, decrease=True)

# Deterministic
fix_all_seeds(config["SEED"])

# Split train/valid
train, valid = df_split(df)

train_loader = make_transistors_loader(train, data)
valid_loader = make_transistors_loader(valid, data)

trainer = Trainer(train_loader, valid_loader, path="./models/LSTM/model_best.bin")
preds, loss, acc = trainer.validate(raw=True)
print(f"loss: {loss} | AUCROC: {acc}")

****************************************************************************************************
The total traning steps is 109400.0.
The warmup steps is 21880.
****************************************************************************************************


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 2188/2188 [04:12<00:00,  8.68it/s]


loss: 203.6854613476741 | AUCROC: 0.7710668766312694


### Dummy model test

In [4]:
## roc auc
from sklearn.metrics import roc_auc_score
roc_auc_score(valid.label, [0]*len(valid.label))
roc_auc_score(valid.label, [1]*len(valid.label))

ValueError: multi_class must be in ('ovo', 'ovr')

In [8]:
## naive accuracy
label_pred = np.argmax(np.mean(preds, axis=1), axis=-1)
### overall accuracy
accuracy = sum(label_pred==valid.label)/len(valid.label)
### all 0 accuracy
all_0 = sum(valid.label==0)/len(valid.label)
print(f"num_sample_0: {sum(valid.label==0)} | num_pred_0: {sum(label_pred==0)}")

### all 1 accuracy
all_1 = sum(valid.label==1)/len(valid.label)
print(f"num_sample_1: {sum(valid.label==1)} | num_pred_1: {sum(label_pred==1)}")

### all 2 accuracy
all_2 = sum(valid.label==2)/len(valid.label)
print(f"num_sample_2: {sum(valid.label==2)} | num_pred_2: {sum(label_pred==2)}")

print(f"accuracy: {accuracy} | all_1 accuracy: {all_1} | all_0 accuracy: {all_0} | all_2 accuracy: {all_2}")#%%

num_sample_0: 279953 | num_pred_0: 148028
num_sample_1: 279952 | num_pred_1: 411877
num_sample_2: 0 | num_pred_2: 0
accuracy: 0.5001455604075692 | all_1 accuracy: 0.49999910699136463 | all_0 accuracy: 0.5000008930086354 | all_2 accuracy: 0.0


In [14]:
np.mean(preds, axis=1)[:, 1]>0.4

array([[0.6203505 , 0.37333894, 0.00631083],
       [0.73627925, 0.16434129, 0.09937956],
       [0.8811319 , 0.05249341, 0.0663745 ],
       ...,
       [0.9023207 , 0.01342692, 0.08425245],
       [0.983878  , 0.0106319 , 0.00548997],
       [0.95358497, 0.03941964, 0.00699546]], dtype=float32)

In [4]:
import torch
ckpt = torch.load("/home/charon/project/nmos_inference/models/LSTM/train_log/uid=1-model_name=nmos_lstm-dataset=nmos-val_epoch_acc=0.552-val_epoch_auc=0.529.ckpt")
ckpt["statistics"] = {"accuracy": 0.552, "auc": 0.529, "loss": 0.0, "val_loss": 0.0}
with open("/home/charon/project/nmos_inference/models/LSTM/train_log/uid=1-model_name=nmos_lstm-dataset=nmos-val_epoch_acc=0.552-val_epoch_auc=0.529.ckpt", "wb") as f:
    torch.save(ckpt, f)
print(ckpt)

{'epoch': 0, 'global_step': 58, 'pytorch-lightning_version': '1.6.1', 'state_dict': OrderedDict([('model.lstm.weight_ih_l0', tensor([[ 0.1661,  0.1822],
        [-0.0565,  0.2095],
        [-0.0428,  0.0414],
        [-0.1037,  0.1678],
        [ 0.1960, -0.1599],
        [ 0.2003,  0.0479],
        [ 0.1630,  0.0570],
        [ 0.1109, -0.0354],
        [ 0.1681,  0.0210],
        [-0.0930,  0.0409],
        [-0.0912, -0.0056],
        [-0.1059,  0.1569],
        [-0.1966, -0.1353],
        [-0.0797, -0.1374],
        [ 0.0306, -0.2215],
        [ 0.2163, -0.2017],
        [ 0.1680,  0.0210],
        [-0.0543,  0.1469],
        [ 0.0321,  0.1775],
        [ 0.0536, -0.0615],
        [ 0.0479, -0.0518],
        [ 0.1289,  0.2005],
        [ 0.1693, -0.0833],
        [ 0.1283,  0.0475],
        [ 0.1029, -0.1276],
        [-0.2218, -0.0709],
        [-0.1651,  0.1806],
        [ 0.0751,  0.1096],
        [ 0.0557, -0.0282],
        [ 0.1718, -0.1813],
        [ 0.0415, -0.1456],
       

In [19]:
ckpt = torch.load("/home/charon/project/nmos_inference/models/LSTM/model_name=nmos_lstm-dataset=nmos-lr=0.001-wd=0.05-batch=64-l1=0.0-l2=0.0-dropout=0.3-opt=Adam-loss=diy-seed=42-epoch=43-val_epoch_acc=0.786-val_epoch_auc=0.851.ckpt")
print(ckpt)

{'epoch': 43, 'global_step': 40304, 'pytorch-lightning_version': '1.6.1', 'state_dict': OrderedDict([('model.lstm.weight_ih_l0', tensor([[ 1.1774e-01,  4.4327e-01],
        [ 3.5233e-01,  7.4056e-01],
        [ 3.8507e-02,  2.9965e-01],
        [ 8.0406e-02,  2.3755e-01],
        [-1.7769e-01, -5.5331e-02],
        [ 8.6296e-02,  9.9325e-02],
        [ 1.1656e-01, -6.7476e-02],
        [ 5.7244e-02,  9.9859e-02],
        [-1.5559e-01,  5.9872e-02],
        [ 2.3380e-01,  2.5980e-01],
        [ 1.3302e-01,  3.8734e-01],
        [ 7.2062e-02,  1.6781e-01],
        [-4.1189e-02,  1.9702e-01],
        [-1.1256e-01,  1.4075e-01],
        [ 2.2718e-01,  2.1543e-01],
        [-7.9831e-03, -2.6189e-02],
        [ 1.8409e-01,  5.5886e-01],
        [-1.0910e-02,  1.9624e-01],
        [-7.6076e-02, -3.2708e-03],
        [-4.6009e-01,  2.9336e-01],
        [-8.6031e-02,  1.9049e-01],
        [ 1.3291e-01,  1.5554e-01],
        [ 3.7880e-01, -2.0721e-01],
        [-1.4588e-01, -2.8291e-01],
       

In [17]:
ckpt["state_dict"].keys()

odict_keys(['model.lstm.weight_ih_l0', 'model.lstm.weight_hh_l0', 'model.lstm.bias_ih_l0', 'model.lstm.bias_hh_l0', 'model.lstm.weight_ih_l1', 'model.lstm.weight_hh_l1', 'model.lstm.bias_ih_l1', 'model.lstm.bias_hh_l1', 'model.linear.weight', 'model.linear.bias'])